# Quick Notes

Training pipeline can be found here: https://www.kaggle.com/ragnar123/ubiquant-tf-training-baseline-with-tpu

In [ ]:
import pandas as pd
import numpy as np
import os
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import mixed_precision
from tensorflow.keras import backend as K
import random
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
import ubiquant

In [ ]:
def build_model(shape):
    def fc_block(x, units):
        x = tf.keras.layers.Dropout(0.35)(x)
        x = tf.keras.layers.Dense(units, activation = 'relu')(x)
        return x

    inp = tf.keras.layers.Input((shape))
    x = fc_block(inp, units = 768)
    x = fc_block(x, units = 384)
    x = fc_block(x, units = 192)
    output = tf.keras.layers.Dense(1, activation = 'linear')(x)
    model = tf.keras.models.Model(inputs = [inp], outputs = [output])
    return model

# Get our features list
features = list(np.load('../input/ubiquant-tf-training-baseline-with-tpu/features.npy'))
corr_features = list(np.load('../input/ubiquant-tf-training-baseline-with-tpu/best_corr.npy'))
# Build 5 models and load 5 fold weights
model1 = build_model(len(features))
model2 = build_model(len(features))
model3 = build_model(len(features))
model4 = build_model(len(features))
model5 = build_model(len(features))
model1.load_weights('../input/ubiquant-tf-training-baseline-with-tpu/simple_fc_dnn_1.h5')
model2.load_weights('../input/ubiquant-tf-training-baseline-with-tpu/simple_fc_dnn_2.h5')
model3.load_weights('../input/ubiquant-tf-training-baseline-with-tpu/simple_fc_dnn_3.h5')
model4.load_weights('../input/ubiquant-tf-training-baseline-with-tpu/simple_fc_dnn_4.h5')
model5.load_weights('../input/ubiquant-tf-training-baseline-with-tpu/simple_fc_dnn_5.h5')
models = [model1, model2, model3, model4, model5]
# Predict
env = ubiquant.make_env()
iter_test = env.iter_test() 
for (test_df, sample_prediction_df) in iter_test:
    for col in corr_features:
        test_df['time_id'] = test_df['row_id'].str[0:4].astype(np.int64)
        mapper = test_df.groupby(['time_id'])[col].mean().to_dict()
        test_df[f'time_id_{col}'] = test_df['time_id'].map(mapper)
    predictions = []
    for model in models:
        predictions.append(model.predict(test_df[features]))
    sample_prediction_df['target'] = np.average(predictions, axis = 0)
    env.predict(sample_prediction_df)